---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [351]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    university_towns = pd.read_table('university_towns.txt', header=None)
    mylist = []
    state = ''
    region = ''

    for value in university_towns[0]:
        if ('[ed' in value):
            state = value.split('[ed')[0]

        else:
            region = value.split('(')[0].strip()
            mylist.append([state, region])
    
    df = pd.DataFrame(mylist, columns=['State', 'RegionName'])
    
    return df

In [352]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    GDP_quarterly = pd.read_excel('gdplev.xls', skiprows=220, header = None, names=['A','B','C','D', 'YearQuarter',
                                                                                'GDP in billions of current dollars',
                                                                               'GDP in billions of chained 2009 dollars','E'])
    
    GDP_quarterly = GDP_quarterly[['YearQuarter', 'GDP in billions of chained 2009 dollars']]
    df = GDP_quarterly['GDP in billions of chained 2009 dollars']
    
    l = []
    for i in range(2,len(df)-1):
        if ((df[i] < df[i-1]) & (df[i-1] < df[i-2])):
            a = GDP_quarterly.loc[(i-1),'YearQuarter']
            l.append(a)
    
    answer = l[0]
    
    return answer

In [353]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    recession_start = get_recession_start()
    GDP_quarterly = pd.read_excel('gdplev.xls', skiprows=220, header = None, names=['A','B','C','D', 'YearQuarter',
                                                                                'GDP in billions of current dollars',
                                                                               'GDP in billions of chained 2009 dollars','E'])
    
    GDP_quarterly = GDP_quarterly[['YearQuarter', 'GDP in billions of chained 2009 dollars']]
    df = GDP_quarterly['GDP in billions of chained 2009 dollars']
    
    l = []
    for i in range(2,len(df)-1):
        if ((df[i] > df[i-1]) & (df[i-1] > df[i-2])):
            a = GDP_quarterly.loc[i,'YearQuarter']
            l.append(a)
    
    final_l=[]
    for i in range(0, len(l)-1):
        if (l[i] > recession_start):
            a = l[i]
            final_l.append(a)
            
    answer = final_l[0]
    
    return answer

In [403]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    
    GDP_quarterly = pd.read_excel('gdplev.xls', skiprows=220, header = None, names=['A','B','C','D', 'YearQuarter',
                                                                                'GDP in billions of current dollars',
                                                                               'GDP in billions of chained 2009 dollars','E'])
    
    GDP_quarterly = GDP_quarterly[['YearQuarter', 'GDP in billions of chained 2009 dollars']]
    
    df = GDP_quarterly[GDP_quarterly['YearQuarter'] <= recession_end]
    df = df[df['YearQuarter'] >= recession_start]
    index = np.argmin(df['GDP in billions of chained 2009 dollars'])
    answer = df.loc[index, 'YearQuarter']
    
    return answer

In [404]:
get_recession_bottom()

'2009q2'

In [373]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    
    housing_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    housing_data = housing_data.replace({'State':states})
    housing_data = housing_data.set_index(['State', 'RegionName'])
    
    dates = pd.date_range('01-2000', periods=200, freq='M')
    dates = pd.PeriodIndex(dates, freq='M')
    cols = dates.strftime('%Y-%m')

    housing_data = housing_data[cols]
    housing_data.columns = pd.to_datetime(housing_data.columns)
    housing_data = housing_data.resample('Q',axis=1).mean()
    housing_data.columns = housing_data.columns.strftime('%Y-%m')
    
    mylist = []
    for date in housing_data.columns:
        if (date.split('-')[1] == '03'):
            mylist.append(date.split('-')[0] + 'q1')
        elif(date.split('-')[1] == '06'):
            mylist.append(date.split('-')[0] + 'q2')
        elif(date.split('-')[1] == '09'):
            mylist.append(date.split('-')[0] + 'q3')
        else:
            mylist.append(date.split('-')[0] + 'q4')

    housing_data.columns = mylist

    return housing_data

In [399]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    df = convert_housing_data_to_quarters()
    univ_towns = get_list_of_university_towns()
    y = get_recession_bottom()
    x = get_recession_start()

    quarterbefore = pd.to_datetime(x) - pd.DateOffset(days = 1)
    quarterbefore = quarterbefore.strftime('%Y-%m')
    
    if (quarterbefore.split('-')[1] == '03'):
        quarterbefore = quarterbefore.split('-')[0] + 'q1'
    elif(quarterbefore.split('-')[1] == '06'):
        quarterbefore = quarterbefore.split('-')[0] + 'q2'
    elif(quarterbefore.split('-')[1] == '09'):
        quarterbefore = quarterbefore.split('-')[0] + 'q3'
    else:
        quarterbefore = quarterbefore.split('-')[0] + 'q4'
    
    price_ratio = df[quarterbefore].div(df[y])
    df['PriceRatio'] = price_ratio

    mylist = univ_towns.to_records(index=False).tolist()
    df_1 = df.loc[mylist]
    df_2 = df.loc[-df.index.isin(mylist)]

    result = ttest_ind(df_1['PriceRatio'], df_2['PriceRatio'], nan_policy='omit')
    pvalue = result[1]

    different = pvalue < 0.01
    p = pvalue
    m1 = df_1['PriceRatio'].mean()
    m2 = df_2['PriceRatio'].mean()

    if m1<m2:
        better = "university town"
    else:
        better = "non-university town"
    
    answer = (different, p, better)
    
    return answer